### Reading parameters
1) Complete a .env file with the model, endpoint and api key that you get from MAAS: https://maas.apps.prod.rhoai.rh-aiservices-bu.com/admin/applications/5585?service_id=10

#### For instance:
MAAS_GRANITE_38="{\"model\": \"granite-3-8b-instruct\", \"apikey\": \"xxxxx\", \"endpoint\": \"https://granite-3-8b-instruct-maas-apicast-production.apps.prod.rhoai.rh-aiservices-bu.com:443\"}"





#todo parameters for langchain

In [1]:
import requests
import urllib3
import numpy as np
import json
import os
from dotenv import load_dotenv



In [2]:
def read_parameter_dict(dict_name):
    load_dotenv()
    return(json.loads(os.getenv(dict_name)))


In [3]:
def read_model_parameters(dict_name):
    load_dotenv()
    par_dict=json.loads(os.getenv(dict_name))
    model=par_dict['model']
    apikey=par_dict['apikey']
    endpoint=par_dict['endpoint']
    return(model, apikey, endpoint)


In [4]:
def request_inference(model_name,endpoint, apikey, prompt,max_token=15, temperature=0):
    # print(f"Model {model_name} Endpoint {endpoint} {prompt}")
    completion = requests.post(
    url=endpoint+'/v1/completions',
    json={
      "model": model_name,
      "prompt": prompt,
      "max_tokens": max_token,
      "temperature": temperature
    },
    headers={'Authorization': 'Bearer '+apikey}).json()
    return completion

In [5]:
from langchain_community.llms import VLLMOpenAI
def langchain_inference(model,endpoint, apikey, prompt):
    llm = VLLMOpenAI(
        openai_api_key=apikey,
        openai_api_base=endpoint+"/v1",
        model_name=model,
        model_kwargs={"stop": ["."]},
    )
    return(llm.invoke(prompt))

## Testing inference granite 3.1-8b-instruct
Model: granite-3-8b-instruct

Hugging Face: https://huggingface.co/ibm-granite/granite-3.1-8b-instruct








In [6]:
granite_38=read_parameter_dict("MAAS_GRANITE_38")
granite_38

{'model': 'granite-3-8b-instruct',
 'apikey': '157356977debf113cb5275dccf0b3f75',
 'endpoint': 'https://granite-3-8b-instruct-maas-apicast-production.apps.prod.rhoai.rh-aiservices-bu.com:443'}

In [7]:
response=request_inference(granite_38['model'],granite_38['endpoint'],granite_38['apikey'], "San Francisco es",50)
response['choices'][0]['text']

' una ciudad que se caracteriza por su diversidad cultural, su vida nocturna vibrante y su impresionante arquitectura. Es una ciudad que ofrece una gran variedad de activ'

In [8]:
response=langchain_inference(granite_38['model'],granite_38['endpoint'],granite_38['apikey'], "Roma es")
response

' una ciudad en constante evolución, y su gastronomía es un reflejo de esa transformación'

### Testing DeepSeek-R1
https://huggingface.co/deepseek-ai/DeepSeek-R1



In [9]:
deepseek_r1=read_parameter_dict("DEEPSEEK_R1")
deepseek_r1

{'model': 'deepseek-r1-distill-qwen-14b',
 'apikey': '4975945900708af1581b7ef430e90b6a',
 'endpoint': 'https://deepseek-r1-distill-qwen-14b-maas-apicast-production.apps.prod.rhoai.rh-aiservices-bu.com:443'}

In [10]:
model=deepseek_r1['model']
endpoint=deepseek_r1['endpoint']
apikey=deepseek_r1['apikey']
prompt="how many rectangles of 10cm x 13 cm enter in a square meter?"
max_tokens=1000

In [11]:
response=request_inference(model, endpoint,apikey, prompt, 500)


In [12]:
response['choices'][0]['text']

" - Brainly.com\\nHow many rectangles of 10cm x 13 cm enter in a square meter?\\nby jessicajoy  |  earlier\\nOkay, so I have this problem here: How many rectangles of 10 cm by 13 cm can fit into a square meter? Hmm, let me think about how to approach this. I guess I need to figure out how many of these rectangles can fit into a meter square. Since the rectangles are in centimeters and the square is in meters, I should probably convert everything to the same unit first. \n\nAlright, a square meter is 100 cm by 100 cm because 1 meter is 100 centimeters. So, the area of the square is 100 cm multiplied by 100 cm, which is 10,000 square centimeters. Each rectangle is 10 cm by 13 cm, so the area of one rectangle is 10 cm multiplied by 13 cm, which is 130 square centimeters. \n\nIf I want to find out how many of these rectangles can fit into the square meter, I can divide the total area of the square by the area of one rectangle. So, that would be 10,000 divided by 130. Let me calculate that.

### Testing Granite-Embedding-278m-multilingual
https://huggingface.co/ibm-granite/granite-embedding-278m-multilingual

In [22]:
emb_model, emb_apikey, emb_endpoint=read_model_parameters("GRANITEEMBEDDING")

In [14]:
print(f"{emb_endpoint}  {emb_apikey} {emb_model}")

'granite-embedding-278m-multilingual'

In [15]:
test_strings = ["That is a happy dog","That is a very happy person","Today is a sunny day"]

embeddings = requests.post(
    url=emb_endpoint+'/v1/embeddings',
    json={"input": test_strings, "model": emb_model},
    headers={'Authorization': 'Bearer '+emb_apikey}
).json()

print(embeddings)

{'id': 'embd-da5437a8637f4387b2353842960833ca', 'object': 'list', 'created': 1739445027, 'model': 'granite-embedding-278m-multilingual', 'data': [{'index': 0, 'object': 'embedding', 'embedding': [-0.041351318359375, -0.01012420654296875, -0.00511932373046875, -0.0018568038940429688, 0.01412200927734375, -0.056640625, 0.05419921875, 0.01367950439453125, 0.03753662109375, 0.01279449462890625, 0.071044921875, 0.0305633544921875, 0.057159423828125, -0.010101318359375, -0.0128326416015625, 0.021087646484375, 0.02960205078125, -0.0149078369140625, 0.02142333984375, 0.0231170654296875, -0.01375579833984375, 0.021026611328125, -0.0297698974609375, 0.073974609375, 0.0197906494140625, 0.0036792755126953125, 0.0161895751953125, 0.040679931640625, -0.029937744140625, 0.05218505859375, -0.0030040740966796875, 0.0219879150390625, 0.001056671142578125, 0.09161376953125, 0.059844970703125, 0.043212890625, -0.0012521743774414062, -0.0187835693359375, -0.043701171875, 0.05609130859375, -0.04611206054687

### Testing Nomic embed-text-v1-5
https://www.nomic.ai/blog/posts/nomic-embed-text-v1



In [16]:
emb_model, emb_apikey, emb_endpoint=read_model_parameters("NAOEMIC")

In [17]:
from langchain_openai import OpenAIEmbeddings


embeddings_model = OpenAIEmbeddings(openai_api_base=emb_endpoint, openai_api_key=emb_apikey, model=emb_model)

test_strings = ["That is a happy dog","That is a very happy person","Today is a sunny day"]



In [18]:
print(f"{emb_endpoint}  {emb_apikey} {emb_model}")

https://nomic-embed-text-v1-5-maas-apicast-production.apps.prod.rhoai.rh-aiservices-bu.com:443  5f046765cda923c2dc06f1768fe1d9f0 /mnt/models


In [19]:
embeddings = embeddings_model.embed_documents(test_strings)

print(embeddings)

NotFoundError: Error code: 404 - {'detail': 'Not Found'}

In [21]:
from langchain_openai import OpenAIEmbeddings

API_URL = "https://nomic-embed-text-v1-5-maas-apicast-production.apps.prod.rhoai.rh-aiservices-bu.com:443"
API_KEY = emb_apikey

embeddings_model = OpenAIEmbeddings(openai_api_base=API_URL, openai_api_key=API_KEY, model=emb_model)

test_strings = ["That is a happy dog","That is a very happy person","Today is a sunny day"]

embeddings = embeddings_model.embed_documents(test_strings)

print(embeddings)

NotFoundError: Error code: 404 - {'detail': 'Not Found'}